<a href="https://colab.research.google.com/github/jads-nl/execute-nhs-proms/blob/master/notebooks/1.1-collect-initial-data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.1 - Collect initial data

## Background
Read up on the documentation of the NHS PROMs data to prepare for this lesson:
- [PROMs Guide](https://github.com/jads-nl/execute-nhs-proms-hko/blob/master/references/nhs/proms_guide_v12.pdf?raw=true)
- [Data Dictionairy](https://github.com/jads-nl/execute-nhs-proms-hko/blob/master/references/nhs/proms_data_dictionary.pdf?raw=true?)

## Objectives
- How to collect data?
- How to handle missing values?
- How to manage memory usage?
- How to rename columns and variables?

## Python skills
- Use `request` library to download files from the internet
- Use `zipfile` to work with zipfiles
- Know difference between getting a `bytes`, `string` and a `file` object for reading and writing data
- How to do string manipulation for replacing columns names
    - How to use [`toolz.functoolz.compose`](https://toolz.readthedocs.io/en/latest/api.html#toolz.functoolz.compose) to compose functions to operate in series
- How to use pandas:
    - for combining downloaded data into one dataframe: [`pd.concat`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html)
    - do some memory optimization: [`df.select_dtypes`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.select_dtypes.html), [`df.astype`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html), [`pd.to_numeric`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_numeric.html)
    - use [`df.rename`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html) for renaming

In [3]:
from io import BytesIO
import requests
import re
from zipfile import ZipFile
import pandas as pd
from toolz.functoolz import compose


CONDITION = 'Knee'
ORG = 'Provider'

def get_zipped_data_urls(condition, org):
    """
    Get zipped datafiles for 3 years for condition and org
    
    condition: choice of ['Knee', 'Hip']
    org: choice of ['Provider', 'CCG']
    """

    assert condition in ['Knee', 'Hip'], 'Condition is one of ["Knee", "Hip"]'
    assert org in ['Provider', 'CCG'], 'Condition is one of ["Provider", "CCG"]'
    return [
        f'https://github.com/jads-nl/execute-nhs-proms-hko/blob/master/data/external/data-pack-2018-19/{condition}%20Replacement%20{org}%201819.csv.zip?raw=true',
        f'https://github.com/jads-nl/execute-nhs-proms-hko/blob/master/data/external/data-pack-2017-18/{condition}%20Replacements%20{org}%201718.csv.zip?raw=true',
        f'https://github.com/jads-nl/execute-nhs-proms-hko/blob/master/data/external/data-pack-2016-17/{condition}%20Replacement%20{org}%201617.csv.zip?raw=true',
    ]

def clean_python_name(s):
    """
    https://gist.github.com/dkapitan/89ff20eeed38e6d9757fef9e09e23c3d
    Method to convert string to clean string for use
    in dataframe column names such :
        i) it complies to python 2.x object name standard:
           (letter|'_')(letter|digit|'_')
        ii) my preference to use lowercase and adhere
            to practice of case-insensitive column names for data
    Based on
    https://stackoverflow.com/questions/3303312/how-do-i-convert-a-string-to-a-valid-variable-name-in-python
    """
    import re

    # Remove leading characters until we find a letter or underscore, and remove trailing spaces
    s = re.sub('^[^a-zA-Z_]+', '', s.strip())

    # Replace invalid characters with underscores
    s = re.sub('[^0-9a-zA-Z_]', '_', s)

    return s.lower()

ModuleNotFoundError: No module named 'requests'

In [2]:
# exmample: getting a single file
urls = get_zipped_data_urls(CONDITION, ORG)
get_ = requests.get(urls[0])
with ZipFile(BytesIO(get_.content)) as zipfile:
  # print(file_.namelist())
    df_ = pd.read_csv(zipfile.open(zipfile.namelist()[0]),
                    na_values=['*'])
df_.head()

,Provider Code,Procedure,Revision Flag,Year,Age Band,Gender,Pre-Op Q Assisted,Pre-Op Q Assisted By,Pre-Op Q Symptom Period,Pre-Op Q Previous Surgery,...,Knee Replacement Post-Op Q Walking,Knee Replacement Post-Op Q Standing,Knee Replacement Post-Op Q Limping,Knee Replacement Post-Op Q Kneeling,Knee Replacement Post-Op Q Work,Knee Replacement Post-Op Q Confidence,Knee Replacement Post-Op Q Shopping,Knee Replacement Post-Op Q Stairs,Knee Replacement Post-Op Q Score,Knee Replacement OKS Post-Op Q Predicted
0,00C,Knee Replacement,0,2018/19,NaN,NaN,1,0,2,2,...,4,3,4,4,4,4,4,2,43.0,35.133457
1,00C,Knee Replacement,0,2018/19,NaN,NaN,1,0,2,2,...,3,2,3,1,2,3,2,3,29.0,26.031412
2,00C,Knee Replacement,0,2018/19,NaN,NaN,2,0,4,2,...,2,2,1,0,2,2,0,2,16.0,26.818595
3,00C,Knee Replacement,0,2018/19,NaN,NaN,2,0,2,2,...,4,3,1,1,3,4,3,3,37.0,33.841701
4,00C,Knee Replacement,0,2018/19,NaN,NaN,2,0,1,1,...,4,4,4,3,4,4,4,4,47.0,38.841248


In [3]:
# assignment: write a loop to get all files
dfs = []
for url in get_zipped_data_urls(CONDITION, ORG):
    get_ = requests.get(url)
    with ZipFile(BytesIO(get_.content)) as zipfile:
        dfs.append(
            pd.read_csv(zipfile.open(zipfile.namelist()[0]), na_values=['*']))
df = pd.concat(dfs)

# reset index since we have duplicate integer indexes from different years
df.reset_index(drop=True, inplace=True)

In [4]:
# check we have 3 years of data
df['Year'].unique()

array(['2018/19', '2017/18', '2016/17'], dtype=object)

In [5]:
# inspect memory usage: orignal is 85.3 MB
df.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136390 entries, 0 to 136389
Columns: 81 entries, Provider Code to Knee Replacement OKS Post-Op Q Predicted
dtypes: float64(8), int64(69), object(4)
memory usage: 84.3+ MB


In [6]:
# reduce memory: strings to categories
# see: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.select_dtypes.html
# see: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].astype('category')
    
# int64 to smallest, unsigned int
for col in df.select_dtypes(include='int64').columns:
    df[col] = pd.to_numeric(df[col], downcast='unsigned')
    
# float64 to smallest, unsigned int
for col in df.select_dtypes(include='float64').columns:
    df[col] = pd.to_numeric(df[col], downcast='float')
    
# memory footprint now less than a fifth: 16 MB
df.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136390 entries, 0 to 136389
Columns: 81 entries, Provider Code to Knee Replacement OKS Post-Op Q Predicted
dtypes: category(4), float32(8), uint16(2), uint32(2), uint8(65)
memory usage: 14.8 MB


In [7]:
# example: common string operations
def shorten_name(s):
    return (s.replace('pre_op_q', 't0')
             .replace('post_op_q', 't1')
             .replace('knee_replacement', 'oks')
             .replace('hip_replacement', 'ohs')
           )

# Note: toolz.functoolz.compose applies functions from right to left
# use compose_left if you want left to right
df.rename(columns=compose(shorten_name, clean_python_name, shorten_name), inplace=True)
df.columns

Index(['provider_code', 'procedure', 'revision_flag', 'year', 'age_band',
       'gender', 't0_assisted', 't0_assisted_by', 't0_symptom_period',
       't0_previous_surgery', 't0_living_arrangements', 't0_disability',
       'heart_disease', 'high_bp', 'stroke', 'circulation', 'lung_disease',
       'diabetes', 'kidney_disease', 'nervous_system', 'liver_disease',
       'cancer', 'depression', 'arthritis', 't0_mobility', 't0_self_care',
       't0_activity', 't0_discomfort', 't0_anxiety', 't0_eq5d_index_profile',
       't0_eq5d_index', 't1_assisted', 't1_assisted_by',
       't1_living_arrangements', 't1_disability', 't1_mobility',
       't1_self_care', 't1_activity', 't1_discomfort', 't1_anxiety',
       't1_satisfaction', 't1_sucess', 't1_allergy', 't1_bleeding', 't1_wound',
       't1_urine', 't1_further_surgery', 't1_readmitted',
       't1_eq5d_index_profile', 't1_eq5d_index',
       'oks_eq_5d_index_t1_predicted', 't0_eq_vas', 't1_eq_vas',
       'oks_eq_vas_t1_predicted', 'oks

In [8]:
# explain how you can use parquet. File 6.3 MB
df.to_parquet(f'../data/interim/{CONDITION.lower()}-{ORG.lower()}.parquet', compression='gzip')